In [135]:
%pip install numpy pandas
%pip install scikit-learn
%pip install xgboost
%pip install category_encoders
%pip install catboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Importing CSVs

In [136]:
import pandas as pd
import numpy as np
import random

# Feature Extraction and Data Cleaning

## Feature Engineering for batsman and bowler

The runs/match is the new metric for a batsman. \
The eco/match is the new metric for a bowler.

If batsman/bowler isnt in the dict, then avg score will be imputed.

In [137]:
batsman_df = pd.read_csv('663e2b548c98c_batsman_level_scorecard.csv')
bowler_df = pd.read_csv('663e2b2c60743_bowler_level_scorecard.csv')

In [138]:
batsman_df[batsman_df["strike_rate"].isnull()] = 110.31

/tmp/ipykernel_35672/2080294716.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '110.31' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batsman_df[batsman_df["strike_rate"].isnull()] = 110.31
/tmp/ipykernel_35672/2080294716.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '110.31' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batsman_df[batsman_df["strike_rate"].isnull()] = 110.31
/tmp/ipykernel_35672/2080294716.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '110.31' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batsman_df[batsman_df["strike_rate"].isnull()] = 110.31
/tmp/ipykernel_35672/2080294716.py:1: Fu

In [139]:
batsman_scores = {}
bowler_scores = {}
strike_rates = {}
avg_wickets = {}

for i, row in batsman_df.iterrows():
    if row['batsman_id'] in batsman_scores:
        batsman_scores[row['batsman_id']][1] += 1
        batsman_scores[row['batsman_id']][0] = (batsman_scores[row['batsman_id']][0]*(batsman_scores[row['batsman_id']][1] - 1) + row['runs']) / batsman_scores[row['batsman_id']][1]
    else:
        batsman_scores[row['batsman_id']] = [row['runs'], 1]

    if row['batsman_id'] in strike_rates:
        strike_rates[row['batsman_id']][1] += 1
        strike_rates[row['batsman_id']][0] = (strike_rates[row['batsman_id']][0]*(strike_rates[row['batsman_id']][1] - 1) + row['strike_rate']) / strike_rates[row['batsman_id']][1]
    else:
        strike_rates[row['batsman_id']] = [row['strike_rate'], 1]


for i, row in bowler_df.iterrows():
    if row['bowler_id'] in bowler_scores:
        bowler_scores[row['bowler_id']][1] += 1
        bowler_scores[row['bowler_id']][0] = (bowler_scores[row['bowler_id']][0]*(bowler_scores[row['bowler_id']][1] - 1) + row['economy']) / bowler_scores[row['bowler_id']][1]
    else:
        bowler_scores[row['bowler_id']] = [row['economy'], 1]

    if row['bowler_id'] in avg_wickets:
        avg_wickets[row['bowler_id']][1] += 1
        avg_wickets[row['bowler_id']][0] = (avg_wickets[row['bowler_id']][0]*(avg_wickets[row['bowler_id']][1] - 1) + row['wicket_count']) / avg_wickets[row['bowler_id']][1]
    else:
        avg_wickets[row['bowler_id']] = [row['wicket_count'], 1]

## Mods to original df


1. Drop season
2. Drop city
3. Expand and drop roster_ids
4. Change toss winner team into its ID
5. Change date time object into crt format and order by date 
6. series_name ? -- Drop?
7. venue ? -- Drop?
8. Maybe remove "team_count_50runs_last15" ?? 

In [140]:
def extract_player_ids(df: pd.DataFrame) -> pd.DataFrame:
    t1_players = df['team1_roster_ids'].str.split(':', expand=True)
    t2_players = df['team2_roster_ids'].str.split(':', expand=True)
    
    # Convert split strings to integers
    t1_players = t1_players.apply(pd.to_numeric, errors='coerce').astype('Int64')
    t2_players = t2_players.apply(pd.to_numeric, errors='coerce').astype('Int64')
    
    # Some cols had extra team members, remove them
    t1_players.drop(columns=11, inplace=True)
    t2_players.drop(columns=11, inplace=True)
    
    t1_players_headings = [f"t1_p{i+1}" for i in range(t1_players.shape[1])]
    t2_players_headings = [f"t2_p{i+1}" for i in range(t2_players.shape[1])]
    
    t1_players.columns = t1_players_headings
    t2_players.columns = t2_players_headings
    df = pd.concat([df, t1_players, t2_players], axis=1)
    df.drop(columns=['team1_roster_ids', 'team2_roster_ids'], inplace=True)

    return df


In [141]:
def extract_batsman_bowler_scores(new_df: pd.DataFrame, batsman_scores: dict, bowler_scores: dict) -> pd.DataFrame: 
    new_df["t1_batsman_scores"] = 0
    new_df["t1_bowler_scores"] = 0
    new_df["t1_strike_rate"] = 0
    new_df["t1_avg_wickets"] = 0

    new_df["t2_batsman_scores"] = 0
    new_df["t2_bowler_scores"] = 0
    new_df["t2_strike_rate"] = 0
    new_df["t2_avg_wickets"] = 0

    new_players = 0

    t1_players = new_df["team1_roster_ids"].str.split(":")
    for i, row in new_df.iterrows():
        for p in t1_players[i]:
            p = float(p)
            if p in batsman_scores.keys():
                new_df.at[i, "t1_batsman_scores"] += batsman_scores[p][0]
            if p in bowler_scores.keys():
                new_df.at[i, "t1_bowler_scores"] += bowler_scores[p][0]
            if p not in batsman_scores.keys() and p not in bowler_scores.keys():
                new_players += 1
                key_bat, val = random.choice(list(batsman_scores.items()))
                key_bowl, val = random.choice(list(bowler_scores.items()))
                new_df.at[i, "t1_batsman_scores"] += batsman_scores[key_bat][0]
                new_df.at[i, "t1_bowler_scores"] += bowler_scores[key_bowl][0]

            if p in strike_rates.keys():
                new_df.at[i, "t1_strike_rate"] += strike_rates[p][0]
            if p in avg_wickets.keys():
                new_df.at[i, "t1_avg_wickets"] += avg_wickets[p][0]
            if p not in strike_rates.keys() and p not in avg_wickets.keys():
                key_str, val = random.choice(list(strike_rates.items()))
                key_wic, val = random.choice(list(avg_wickets.items()))
                new_df.at[i, "t1_strike_rate"] += strike_rates[key_str][0]
                new_df.at[i, "t1_avg_wickets"] += avg_wickets[key_wic][0]
    
    t2_players = new_df["team2_roster_ids"].str.split(":")
    for i, row in new_df.iterrows():
        for p in t2_players[i]:
            p = float(p)
            if p in batsman_scores.keys():
                new_df.at[i, "t2_batsman_scores"] += batsman_scores[p][0]
            if p in bowler_scores.keys():
                new_df.at[i, "t2_bowler_scores"] += bowler_scores[p][0]
            if p not in batsman_scores.keys() and p not in bowler_scores.keys():
                new_players += 1
                key_bat, val = random.choice(list(batsman_scores.items()))
                key_bowl, val = random.choice(list(bowler_scores.items()))
                new_df.at[i, "t2_batsman_scores"] += batsman_scores[key_bat][0]
                new_df.at[i, "t2_bowler_scores"] += bowler_scores[key_bowl][0]
            
            if p in strike_rates.keys():
                new_df.at[i, "t2_strike_rate"] += strike_rates[p][0]
            if p in avg_wickets.keys():
                new_df.at[i, "t2_avg_wickets"] += avg_wickets[p][0]
            if p not in strike_rates.keys() and p not in avg_wickets.keys():
                key_str, val = random.choice(list(strike_rates.items()))
                key_wic, val = random.choice(list(avg_wickets.items()))
                new_df.at[i, "t2_strike_rate"] += strike_rates[key_str][0]
                new_df.at[i, "t2_avg_wickets"] += avg_wickets[key_wic][0]

    new_df.drop(columns=['team1_roster_ids', 'team2_roster_ids'], inplace=True)

    return new_df

In [142]:
def change_date_time(df: pd.DataFrame) -> pd.DataFrame:
    df['match_dt'] = pd.to_datetime(df['match_dt'], format='%Y-%m-%d')
    df = df.sort_values(by='match_dt')
    df.reset_index(drop=True, inplace=True)

    return df

In [143]:
def add_toss_winner_id(df: pd.DataFrame) -> pd.DataFrame:
    df["toss winner id"] = None  # initialize the column with None values
    for i, row in df.iterrows():
        if row['toss winner'] == row['team1']:
            df.at[i, 'toss winner id'] = row['team1_id']
        else:
            df.at[i, 'toss winner id'] = row['team2_id']

    df["toss winner id"] = df["toss winner id"].astype(int)

    df.drop(columns=['toss winner'], inplace=True)

    return df

In [144]:
def encode_winner_id(df: pd.DataFrame) -> pd.DataFrame:
    # 1- team1, 0 - team2
    df["winning team"] = None
    for i, row in df.iterrows():
        if row['winner_id'] == row['team1_id']:
            df.at[i, 'winning team'] = 1
        else:
            df.at[i, 'winning team'] = 0
    
    df["winning team"] = df["winning team"].astype(int)

    df.drop(columns=['winner_id'], inplace=True)

    return df


In [145]:
def drop_redun_cols(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(columns=['season', 'city', 'series_name', 'venue', 'team1', 'team2', 'winner',], inplace=True)
    
    return df

In [146]:
new_df = pd.read_csv("663e2b6d54457_train_data_with_samplefeatures.csv")
new_df

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,...,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,...,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.00,157.178571
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,...,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.00,103.500000
2,9433269,We,10576,3298427.0:2288789.0:7773338.0:3519011.0:368195...,Ne,8987,4003390.0:1749075.0:1626526.0:4172447.0:551672...,We,10576,Ne,...,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.00,154.333333
3,9587073,Ga An Ws,36084,8127230.0:4690328.0:4069666.0:7960847.0:469018...,Bs Rs,36070,3462080.0:2436405.0:1798705.0:7550857.0:574247...,Ga An Ws,36084,Ga An Ws,...,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.00,144.250000
4,9516457,Pb Ks,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082...,Gt Ts,48341,1958683.0:7491224.0:8059029.0:4377610.0:225245...,Gt Ts,48341,Gt Ts,...,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.00,189.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9128601,Pb Ks,30407,2789079.0:197658.0:2398346.0:2827327.0:2082044...,Ci Sr Ks,30414,7422673.0:249087.0:3519011.0:5001170.0:232000....,Pb Ks,30407,Ci Sr Ks,...,2022-04-25,night match,In Pr Le,2022,5004,0.823529,1.000000,147.333333,66.67,166.400000
944,9433241,Mx,8700,4421689.0:7752989.0:1941743.0:4489722.0:767287...,St,9701,6139370.0:7694581.0:3294444.0:3239102.0:632036...,St,9701,St,...,2023-06-02,day/night match,Vy Bt,2023,1042,1.571429,0.012346,167.400000,0.00,170.466667
945,9097227,Bd,22497,4239773.0:1941743.0:3007969.0:4172972.0:155625...,Wn Pe,23869,323049.0:4876122.0:4164978.0:1837205.0:3373138...,Wn Pe,23869,Bd,...,2022-02-07,day match,CA T0 Ce,2021/22,1224,3.000000,1.000000,NaN,0.00,NaN
946,9516695,Rn Rs,30428,8058959.0:2162782.0:2981614.0:4690188.0:212569...,Ss Hd,36014,5958840.0:7491294.0:3127354.0:3057312.0:420349...,Ss Hd,36014,Rn Rs,...,2023-05-07,night match,In Pr Le,2023,4661,0.789474,1.487805,182.800000,66.67,133.375000


In [147]:
new_df = extract_batsman_bowler_scores(new_df, batsman_scores, bowler_scores)
new_df = encode_winner_id(new_df)
new_df = add_toss_winner_id(new_df)
new_df = change_date_time(new_df)
new_df = drop_redun_cols(new_df)
new_df

/tmp/ipykernel_35672/4121842941.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.333333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.at[i, "t1_batsman_scores"] += batsman_scores[p][0]
/tmp/ipykernel_35672/4121842941.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '95.55666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.at[i, "t1_strike_rate"] += strike_rates[p][0]
/tmp/ipykernel_35672/4121842941.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.8295' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.at[i, "t1_bowler_scores"] += bowler_scores[key_bowl][0]
/tmp/ipy

,match id,team1_id,team2_id,toss decision,match_dt,lighting,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,...,t1_batsman_scores,t1_bowler_scores,t1_strike_rate,t1_avg_wickets,t2_batsman_scores,t2_bowler_scores,t2_strike_rate,t2_avg_wickets,winning team,toss winner id
0,8887675,33921,33928,field,2022-01-01,day/night match,6348,0.583333,0.672131,166.933333,...,164.990940,64.865668,1194.100203,6.394161,180.486717,59.819847,1304.584359,7.577197,1,33928
1,8887689,33935,33949,bat,2022-01-02,day/night match,2932,1.222222,1.327869,178.733333,...,175.942939,65.563866,1218.184305,7.299456,145.693795,66.384949,970.065791,5.588492,1,33935
2,8887703,33949,33942,bat,2022-01-03,night match,440,0.400000,41.000000,171.066667,...,131.260843,65.224949,931.504294,6.255158,178.585387,72.468163,1235.283317,6.164422,0,33949
3,8887717,33928,33914,field,2022-01-05,night match,930,1.000000,61.000000,173.333333,...,209.260752,64.958383,1485.925010,5.575485,160.236141,49.645070,1199.556425,5.613890,0,33914
4,8887731,33921,33942,field,2022-01-06,night match,3961,0.363636,1.952381,167.000000,...,160.291849,71.456329,1049.869226,4.664557,172.057609,77.495298,1297.596095,6.607178,0,33942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9717504,48,188,field,2023-12-27,night match,3184,1.000000,0.603960,177.400000,...,169.139497,82.523262,1232.959554,8.829658,163.182191,74.005587,1067.817523,7.523586,0,188
944,9866373,18360,18570,bat,2023-12-28,day/night match,5382,1.400000,1.487805,165.733333,...,146.101642,67.486819,1156.447504,4.648109,125.040873,53.970643,940.841270,9.051250,0,18360
945,9702783,33928,33949,field,2023-12-28,night match,6348,1.444444,1.000000,175.933333,...,181.753400,58.566689,1211.397711,6.559572,176.226206,77.188414,1227.084980,6.036222,0,33949
946,9702790,33914,33942,bat,2023-12-29,night match,8252,0.529412,1.000000,162.133333,...,182.642780,62.116303,1221.738988,6.419748,179.579465,50.631640,1148.771569,6.072154,0,33914


In [148]:
new_df["t2_strike_rate"].isnull().sum()

np.int64(0)

## Finding Historical Relationship

In [149]:
match_data = pd.read_csv("664389efa0868_match_level_scorecard.csv")
match_data = change_date_time(match_data)
match_data

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,...,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,...,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0...,18570,2653993.0:6718326.0:6718382.0:2486896.0:228878...,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,...,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1...,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652....,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,...,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:306369...,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803....,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,...,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0...,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498...,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,...,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948...,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127...,other_domestic,33928,3125849.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,9717504,Nw Zd,Bh,Bh,wickets,5.0,Bh,field,Mn Pk Nr,Napier,...,137.0,5.0,114.0,48,6718326.0:4381761.0:2672214.0:5764576.0:306995...,188,3758565.0:2104332.0:4403419.0:3056752.0:693023...,international,188,5831622.0
1685,9866373,Oo,Wn,Wn,wickets,9.0,Oo,bat,Mx Pk Aa,Alexandra,...,53.0,1.0,29.0,18360,2319638.0:8444855.0:4172706.0:7907444.0:631714...,18570,9036516.0:8271969.0:3913447.0:9523954.0:835478...,other_domestic,18570,6718410.0
1686,9702783,Ht Hs,Me Ss,Me Ss,wickets,7.0,Me Ss,field,Be Ol Ht,Hobart,...,67.0,3.0,42.0,33928,1611364.0:5843200.0:5716402.0:3890963.0:194364...,33949,5406540.0:2275195.0:2275097.0:3901078.0:796099...,other_domestic,33949,2275195.0
1687,9702790,Ae Ss,Me Rs,Me Rs,wickets,4.0,Ae Ss,bat,Ds Sm Me,Melbourne,...,178.0,5.0,115.0,33914,4239038.0:2161599.0:2286472.0:5469008.0:368195...,33942,4003390.0:2654014.0:8176356.0:46794.0:1635773....,other_domestic,33942,8176356.0


In [150]:
def update_history(new_df: pd.DataFrame) -> pd.DataFrame:
    new_df["t1_points"] = 0
    new_df["t2_points"] = 0

    for i, row in new_df.iterrows():
        c_t1_id = row['team1_id']
        c_t2_id = row['team2_id']
        match_date = row['match_dt']
        
        for j, row2 in match_data.iterrows():
            if row2['match_dt'] > match_date:
                break
            
            if row2['team1_id'] == c_t1_id and row2['team2_id'] == c_t2_id:
                if row2['winner_id'] == c_t1_id:
                    new_df.at[i, 't1_points'] += 1
                elif row2['winner_id'] == c_t2_id:
                    new_df.at[i, 't2_points'] += 1

    return new_df

In [151]:
new_df = update_history(new_df)
new_df

,match id,team1_id,team2_id,toss decision,match_dt,lighting,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,...,t1_strike_rate,t1_avg_wickets,t2_batsman_scores,t2_bowler_scores,t2_strike_rate,t2_avg_wickets,winning team,toss winner id,t1_points,t2_points
0,8887675,33921,33928,field,2022-01-01,day/night match,6348,0.583333,0.672131,166.933333,...,1194.100203,6.394161,180.486717,59.819847,1304.584359,7.577197,1,33928,1,0
1,8887689,33935,33949,bat,2022-01-02,day/night match,2932,1.222222,1.327869,178.733333,...,1218.184305,7.299456,145.693795,66.384949,970.065791,5.588492,1,33935,2,0
2,8887703,33949,33942,bat,2022-01-03,night match,440,0.400000,41.000000,171.066667,...,931.504294,6.255158,178.585387,72.468163,1235.283317,6.164422,0,33949,0,2
3,8887717,33928,33914,field,2022-01-05,night match,930,1.000000,61.000000,173.333333,...,1485.925010,5.575485,160.236141,49.645070,1199.556425,5.613890,0,33914,0,1
4,8887731,33921,33942,field,2022-01-06,night match,3961,0.363636,1.952381,167.000000,...,1049.869226,4.664557,172.057609,77.495298,1297.596095,6.607178,0,33942,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9717504,48,188,field,2023-12-27,night match,3184,1.000000,0.603960,177.400000,...,1232.959554,8.829658,163.182191,74.005587,1067.817523,7.523586,0,188,6,3
944,9866373,18360,18570,bat,2023-12-28,day/night match,5382,1.400000,1.487805,165.733333,...,1156.447504,4.648109,125.040873,53.970643,940.841270,9.051250,0,18360,0,4
945,9702783,33928,33949,field,2023-12-28,night match,6348,1.444444,1.000000,175.933333,...,1211.397711,6.559572,176.226206,77.188414,1227.084980,6.036222,0,33949,2,1
946,9702790,33914,33942,bat,2023-12-29,night match,8252,0.529412,1.000000,162.133333,...,1221.738988,6.419748,179.579465,50.631640,1148.771569,6.072154,0,33914,2,3


In [152]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   match id                   948 non-null    int64         
 1   team1_id                   948 non-null    int64         
 2   team2_id                   948 non-null    int64         
 3   toss decision              948 non-null    object        
 4   match_dt                   948 non-null    datetime64[ns]
 5   lighting                   948 non-null    object        
 6   ground_id                  948 non-null    int64         
 7   team_count_50runs_last15   948 non-null    float64       
 8   team_winp_last5            948 non-null    float64       
 9   team1only_avg_runs_last15  927 non-null    float64       
 10  team1_winp_team2_last15    948 non-null    float64       
 11  ground_avg_runs_last15     895 non-null    float64       
 12  t1_batsm

In [153]:
new_df['winning team'].unique()

array([1, 0])

# Encoding

In [154]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import category_encoders as ce

In [155]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   match id                   948 non-null    int64         
 1   team1_id                   948 non-null    int64         
 2   team2_id                   948 non-null    int64         
 3   toss decision              948 non-null    object        
 4   match_dt                   948 non-null    datetime64[ns]
 5   lighting                   948 non-null    object        
 6   ground_id                  948 non-null    int64         
 7   team_count_50runs_last15   948 non-null    float64       
 8   team_winp_last5            948 non-null    float64       
 9   team1only_avg_runs_last15  927 non-null    float64       
 10  team1_winp_team2_last15    948 non-null    float64       
 11  ground_avg_runs_last15     895 non-null    float64       
 12  t1_batsm

In [156]:
new_df

,match id,team1_id,team2_id,toss decision,match_dt,lighting,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,...,t1_strike_rate,t1_avg_wickets,t2_batsman_scores,t2_bowler_scores,t2_strike_rate,t2_avg_wickets,winning team,toss winner id,t1_points,t2_points
0,8887675,33921,33928,field,2022-01-01,day/night match,6348,0.583333,0.672131,166.933333,...,1194.100203,6.394161,180.486717,59.819847,1304.584359,7.577197,1,33928,1,0
1,8887689,33935,33949,bat,2022-01-02,day/night match,2932,1.222222,1.327869,178.733333,...,1218.184305,7.299456,145.693795,66.384949,970.065791,5.588492,1,33935,2,0
2,8887703,33949,33942,bat,2022-01-03,night match,440,0.400000,41.000000,171.066667,...,931.504294,6.255158,178.585387,72.468163,1235.283317,6.164422,0,33949,0,2
3,8887717,33928,33914,field,2022-01-05,night match,930,1.000000,61.000000,173.333333,...,1485.925010,5.575485,160.236141,49.645070,1199.556425,5.613890,0,33914,0,1
4,8887731,33921,33942,field,2022-01-06,night match,3961,0.363636,1.952381,167.000000,...,1049.869226,4.664557,172.057609,77.495298,1297.596095,6.607178,0,33942,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,9717504,48,188,field,2023-12-27,night match,3184,1.000000,0.603960,177.400000,...,1232.959554,8.829658,163.182191,74.005587,1067.817523,7.523586,0,188,6,3
944,9866373,18360,18570,bat,2023-12-28,day/night match,5382,1.400000,1.487805,165.733333,...,1156.447504,4.648109,125.040873,53.970643,940.841270,9.051250,0,18360,0,4
945,9702783,33928,33949,field,2023-12-28,night match,6348,1.444444,1.000000,175.933333,...,1211.397711,6.559572,176.226206,77.188414,1227.084980,6.036222,0,33949,2,1
946,9702790,33914,33942,bat,2023-12-29,night match,8252,0.529412,1.000000,162.133333,...,1221.738988,6.419748,179.579465,50.631640,1148.771569,6.072154,0,33914,2,3


# Splitting Train, Test Data

In [157]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = new_df[new_df['match_dt'] < '2023-05-07']
test_df = new_df[new_df['match_dt'] >= '2023-05-07']

In [158]:
y_train = train_df['winning team']
y_test = test_df['winning team']
X_train = train_df.drop(columns=['winning team']) 
X_test = test_df.drop(columns=['winning team'])

In [159]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 664 entries, 0 to 663
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   match id                   664 non-null    int64         
 1   team1_id                   664 non-null    int64         
 2   team2_id                   664 non-null    int64         
 3   toss decision              664 non-null    object        
 4   match_dt                   664 non-null    datetime64[ns]
 5   lighting                   664 non-null    object        
 6   ground_id                  664 non-null    int64         
 7   team_count_50runs_last15   664 non-null    float64       
 8   team_winp_last5            664 non-null    float64       
 9   team1only_avg_runs_last15  647 non-null    float64       
 10  team1_winp_team2_last15    664 non-null    float64       
 11  ground_avg_runs_last15     620 non-null    float64       
 12  t1_batsman_sc

# Creating Model

In [160]:
def process_datetime(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['year'] = df['match_dt'].dt.year
    df['month'] = df['match_dt'].dt.month
    df['day'] = df['match_dt'].dt.day
    df = df.drop('match_dt', axis=1)
    
    return df

In [161]:
all_columns = X_train.columns.tolist()

categorical_features = ['toss decision', 'lighting']
categorical_transformer = Pipeline(steps=[
    ('encoder', ce.OrdinalEncoder(cols=categorical_features))
])

cols_to_ignore = ['match_dt', 'team1_id', 'team2_id', 'match id', 'ground_id', 'toss winner id']
numerical_features = [col for col in all_columns if col not in categorical_features and col not in cols_to_ignore]
# numerical_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())
# ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('datetime', FunctionTransformer(process_datetime, validate=False), ['match_dt'])
    ])

# Configure XGBClassifier with hyperparameters
xgb_clf = xgb.XGBClassifier()

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_clf)
])

In [162]:
params = {
    "classifier__learning_rate": [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4],
    "classifier__max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
    "classifier__min_child_weight": [1, 3, 5, 7],
    "classifier__gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    "classifier__colsample_bytree": [0.3, 0.4, 0.5, 0.7],
    "classifier__n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

In [163]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
random_search = RandomizedSearchCV(clf, param_distributions=params, n_iter=5, cv=tscv, verbose=3, n_jobs=2, scoring='roc_auc', random_state=42)

# Training Model

In [164]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 2/5] END classifier__colsample_bytree=0.3, classifier__gamma=0.2, classifier__learning_rate=0.9, classifier__max_depth=5, classifier__min_child_weight=7, classifier__n_estimators=600;, score=0.961 total time=   0.1s
[CV 1/5] END classifier__colsample_bytree=0.3, classifier__gamma=0.2, classifier__learning_rate=0.9, classifier__max_depth=5, classifier__min_child_weight=7, classifier__n_estimators=600;, score=0.939 total time=   0.1s
[CV 3/5] END classifier__colsample_bytree=0.3, classifier__gamma=0.2, classifier__learning_rate=0.9, classifier__max_depth=5, classifier__min_child_weight=7, classifier__n_estimators=600;, score=0.976 total time=   0.0s
[CV 4/5] END classifier__colsample_bytree=0.3, classifier__gamma=0.2, classifier__learning_rate=0.9, classifier__max_depth=5, classifier__min_child_weight=7, classifier__n_estimators=600;, score=0.940 total time=   0.1s
[CV 5/5] END classifier__colsample_bytree=0.3, classifier__g

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               'passthrough',
                                                                               ['team_count_50runs_last15',
                                                                                'team_winp_last5',
                                                                                'team1only_avg_runs_last15',
                                                                                'team1_winp_team2_last15',
                                                                                'ground_avg_runs_last15',
                                                                                't1_batsman_scores',
                                                                                't1_bowler_scores',...
                                                                         0.5,
                                                                         0.7],
                                        'classifier__gamma': [0.0, 0.1, 0.2,
                                                              0.3, 0.4],
                                        'classifier__learning_rate': [0.05, 0.1,
                                                                      0.15, 0.2,
                                                                      0.25, 0.3,
                                                                      0.35, 0.4,
                                                                      0.45, 0.5,
                                                                      0.6, 0.7,
                                                                      0.8, 0.9,
                                                                      1, 2, 3,
                                                                      4],
                                        'classifier__max_depth': [3, 4, 5, 6, 8,
                                                                  10, 12, 15],
                                        'classifier__min_child_weight': [1, 3,
                                                                         5, 7],
                                        'classifier__n_estimators': [100, 200,
                                                                     300, 400,
                                                                     500, 600,
                                                                     700, 800,
                                                                     900,
                                                                     1000]},
                   random_state=42, scoring='roc_auc', verbose=3)

In [165]:
# Best parameters and score
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)

# Evaluate on test set
test_score = random_search.score(X_test, y_test)
print("Test set score: ", test_score)

Best parameters found:  {'classifier__n_estimators': 500, 'classifier__min_child_weight': 3, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.05, 'classifier__gamma': 0.3, 'classifier__colsample_bytree': 0.7}
Best cross-validation score:  0.9721402783383477
Test set score:  0.9724042088544769


In [166]:
random_search.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['team_count_50runs_last15',
                                                   'team_winp_last5',
                                                   'team1only_avg_runs_last15',
                                                   'team1_winp_team2_last15',
                                                   'ground_avg_runs_last15',
                                                   't1_batsman_scores',
                                                   't1_bowler_scores',
                                                   't1_strike_rate',
                                                   't1_avg_wickets',
                                                   't2_batsman_scores',
                                                   't2_bowler_scores',
                                                   't2_strike_rate',
                                                   't2_avg_wic...
                               feature_types=None, gamma=0.3, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None, min_child_weight=3,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=500,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=None, ...))])

In [167]:
xgb_clf = xgb.XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.3, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=np.nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None)

In [168]:
# xgb_clf = random_search.best_estimator_


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_clf)
])

In [169]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['team_count_50runs_last15',
                                                   'team_winp_last5',
                                                   'team1only_avg_runs_last15',
                                                   'team1_winp_team2_last15',
                                                   'ground_avg_runs_last15',
                                                   't1_batsman_scores',
                                                   't1_bowler_scores',
                                                   't1_strike_rate',
                                                   't1_avg_wickets',
                                                   't2_batsman_scores',
                                                   't2_bowler_scores',
                                                   't2_strike_rate',
                                                   't2_avg_wic...
                               feature_types=None, gamma=0.3, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None, min_child_weight=3,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=500,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=None, ...))])

In [170]:
y_pred = clf.predict(X_test)
y_pred

array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0])

# Model Evaluation

In [171]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.9014084507042254
Precision: 0.901805197381472
Recall: 0.9014084507042254
F1 Score: 0.901428012519562
Confusion Matrix:
[[130  16]
 [ 12 126]]


# Feature Importance

In [172]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 664 entries, 0 to 663
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   match id                   664 non-null    int64         
 1   team1_id                   664 non-null    int64         
 2   team2_id                   664 non-null    int64         
 3   toss decision              664 non-null    object        
 4   match_dt                   664 non-null    datetime64[ns]
 5   lighting                   664 non-null    object        
 6   ground_id                  664 non-null    int64         
 7   team_count_50runs_last15   664 non-null    float64       
 8   team_winp_last5            664 non-null    float64       
 9   team1only_avg_runs_last15  647 non-null    float64       
 10  team1_winp_team2_last15    664 non-null    float64       
 11  ground_avg_runs_last15     620 non-null    float64       
 12  t1_batsman_sc

In [173]:
classifier = clf.named_steps['classifier']

feature_importances = classifier.feature_importances_

feature_names = numerical_features + categorical_features + ['year', 'month', 'day']

pd.DataFrame(
    data=feature_importances,
    index=feature_names,
    columns=["Feature Importance"]
).sort_values(by="Feature Importance", ascending=False)


,Feature Importance
t1_points,0.355441
t2_points,0.217458
team1_winp_team2_last15,0.097582
t1_avg_wickets,0.038877
t1_batsman_scores,0.030798
team_winp_last5,0.029138
team1only_avg_runs_last15,0.021715
t2_avg_wickets,0.020999
t1_strike_rate,0.020478
t2_strike_rate,0.019601


# Saving Model

In [ ]:
import joblib

joblib_file = 'xgb_pipeline_model.pkl'
joblib.dump(clf, joblib_file)
